# Import Data

In [526]:
###import neccessary packages
from lda import guidedlda
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
from nltk.stem.porter import *
import os
import docx2txt
import re
import sklearn.feature_extraction
stemmer = PorterStemmer()
newseed_words = pd.read_csv('transcript_seedwords1.csv')
newseed_words = newseed_words.dropna()

###below is the backup old transcript table

# text2 = pd.read_csv('Frey transcripts.csv')


In [569]:
###loads file content into transcript_lists###
transcript_list = []

###file path to locate raw_transcripts folder... make sure file path is correct when running###
directory = '/Users/lubasisikwibele/Frey/Final guided lda/raw_transcript'
 
# iterate over files in that directory
for filename in os.listdir(directory):
    my_text = docx2txt.process(directory + '/' + str(filename))
    transcript_list.append(my_text)

In [548]:
###converts list to dataframe###
text = pd.DataFrame(transcript_list, columns=['raw transcripts'])

In [549]:
# text

In [550]:
###locates the header of the file content and uses that to name the file###
def head(header):
    actual_header = re.split('\n', header)
    # empty_header.append(actual_header[0])
    return (actual_header[0])

In [551]:
### creates a new column in the text data frame and loads the filename in correspondence with the raw transcripts###
text['transcript name'] = [head(i) for i in df['transcripts']]

In [532]:
text

,raw transcripts,transcript name
0,Bow Knot - BRICK 2.mp4\n\nLindsey: [00:00:03] ...,Bow Knot - BRICK 2.mp4
1,On the rocks - Night rain - BRICK 0.mp4\n\nLar...,On the rocks - Night rain - BRICK 0.mp4
2,9-26 Evening Hanging-out - BRICK.mp4\n\nSpeake...,9-26 Evening Hanging-out - BRICK.mp4
3,Scene 1\n\n\n\nElizabeth: [00:00:12] I ended u...,Scene 1
4,Day before Wedding - BRICK 2.mp4\n\nLindsey: [...,Day before Wedding - BRICK 2.mp4
5,ROER - BRICK 1.mp4\n\nSpeaker1: [00:00:03] I'l...,ROER - BRICK 1.mp4
6,Home is a Place for Second Chances- BRICK.mp4\...,Home is a Place for Second Chances- BRICK.mp4
7,Wedding Day - BRICK 2.mp4\n\nSpeaker1: [00:00:...,Wedding Day - BRICK 2.mp4
8,MD Anderson Day BRICK 3.2.mp4\n\nSpeaker1: [00...,MD Anderson Day BRICK 3.2.mp4
9,Scene 6\n\n\n\nMaynard: [00:26:28] I noticed w...,Scene 6


In [552]:
###dictionary to change numeric values to relevant category###
seedkeys = {'Faith':0, 'Cancer':1, 'Nature':2, 'Mindfullness':3}

In [553]:
###function that converts numeric value to relevant key. E.g 0 returns 'Faith'###
def get_key(val):
    for key, value in seedkeys.items():
        if val == value:
            return key
 
    return "key doesn't exist"
 

In [554]:
###creates dictionary of seed words from seed words csv file e.g Faith: god,hope...###
seed_dict = newseed_words.to_dict('list')

In [299]:
# seed_dict

In [555]:
###this section has a function that stems seed words and creates a new stemmed dictionary seed words###
def stemList(l):
    return([stemmer.stem(word) for word in l])

#the following creates a new dictionary where stemming has been applied to the values

stemmed_seed_dict = {}
for key in seed_dict:
    for word in key:
        # print(type(word))
        stemmed_seed_dict[key] = stemList(seed_dict[key])

In [536]:
seedlist = []

In [537]:
###creates a final seed words list just like the backup####
for key in stemmed_seed_dict:
    seedlist.append(stemmed_seed_dict[key])
# stemmed_seed_dict['Faith']


In [556]:
###!!!!only for old transcript csv files!!!###

# # transcript_seed_words = transcript_seed_words.dropna()
# text2 = text2.dropna()
# text2 = text2.drop(columns="url")
# text2.rename(columns = {'tweets':'transcripts', 'Company':'transcript_name'}, inplace = True)

# Data Cleaning
1. tokenization
2. remove @users, hashtag symbols, Urls, and special symbols(i.e., '&amp'), non-alphabetic characters, and words that have less than 3 characters
3. remove stopwords
4. lowercase transformation
5. stemming

In [572]:
import gensim
import gensim.corpora as corpora
import re
porter = PorterStemmer()
stop_words = stopwords.words('english')
b = []
for i,u in text.iterrows():
    a = []
    word =''
    for words in str(u['raw transcripts']).split(): #tokenization
        if '@' not in words: #remove @users
            words = words.replace('#','') #remove hashtag symbol
            if '#' not in words:
                if 'http' not in words: #remove URLs
                    if'&amp' not in words: #remove symbol
                        words = re.sub(r'[^a-zA-Z]', ' ', words)#remove non-alphabetic characters
                        if len(words)>2:
                            word += (words+' ')
    doc = ''
    for token in word.split():
        if len(token) >2: # remove words that have less than 3 characters
            token = token.lower()# lowercase form
            if token not in stop_words:# remove stopwords
                token = porter.stem(token) #stemming
                doc += (token+' ')
    b.append(doc)
text['processed']=[i for i in b]

# exclude words that are not in English
non_english_list = ['temiz','rkiy','erik','nda','konu','dan','da','ba','al','viand','para','na','dann','uft','laboratorio','dieser','kalbimi',
                   'restoranda','evento','komo','ind','tica','futuro','sonra','yla','cre','ili','daki',
                   'zaman']
index_axis = []
for index,i in text.iterrows():
    if len(i['processed']) == 0:
        index_axis.append(index)
    else:
        for word in i['processed'].split():
            if word in non_english_list:
                index_axis.append(index)
                break
text.drop(text.index[index_axis],inplace=True)
print("number of tweets after cleaning:",len(text))

number of tweets after cleaning: 30


In [539]:
text

,raw transcripts,transcript name,processed
0,Bow Knot - BRICK 2.mp4\n\nLindsey: [00:00:03] ...,Bow Knot - BRICK 2.mp4,bow knot brick lindsey readi order custom gues...
1,On the rocks - Night rain - BRICK 0.mp4\n\nLar...,On the rocks - Night rain - BRICK 0.mp4,rock night rain brick lar tnt guid hey guy goo...
2,9-26 Evening Hanging-out - BRICK.mp4\n\nSpeake...,9-26 Evening Hanging-out - BRICK.mp4,even hang brick speaker got bean pepper onion ...
3,Scene 1\n\n\n\nElizabeth: [00:00:12] I ended u...,Scene 1,scene elizabeth end nine hour surgeri two mont...
4,Day before Wedding - BRICK 2.mp4\n\nLindsey: [...,Day before Wedding - BRICK 2.mp4,day wed brick lindsey second ryan yeah lindsey...
5,ROER - BRICK 1.mp4\n\nSpeaker1: [00:00:03] I'l...,ROER - BRICK 1.mp4,roer brick speaker check radar go storm come a...
6,Home is a Place for Second Chances- BRICK.mp4\...,Home is a Place for Second Chances- BRICK.mp4,home place second chanc brick speaker big one ...
7,Wedding Day - BRICK 2.mp4\n\nSpeaker1: [00:00:...,Wedding Day - BRICK 2.mp4,wed day brick speaker ye sun shine bright good...
8,MD Anderson Day BRICK 3.2.mp4\n\nSpeaker1: [00...,MD Anderson Day BRICK 3.2.mp4,anderson day brick speaker hear thunder awesom...
9,Scene 6\n\n\n\nMaynard: [00:26:28] I noticed w...,Scene 6,scene maynard notic water reflect light onto s...


# GuidedLDA Model
latent topics are identified from the bigrams shown above.

In [312]:
#back up seed words
# seed_words = [['god','hope','pray','believ','trust','church','convict','optim','religion','ideaolog','confid','group','relationship',
#       'mass','synagogu','anticip','futur','strong','lord','readi','wait','focu','posit','plan','support',
#       'desir','accept','loyal','truth','assent','assur','constant','credenc','credul','depend','fealti',
#         'relianc','sure','sureti','troth', 
#         'hope', 'chang', 'brighter','gratitud','faith', 'opportun', 'love', 'pray', 'hope', 'likelihood'],#0 Faith
#       ['tumor','radiolog','chemotherapi','benign','invas','mastectomi','surgeri','malign','metastasi','melanoma',
#        'carcigen','cancer','precancer','survivor','diseas','sick','ill','spread','lump','cell','grow','neck','bodi',
#        'organ','lung','heal','emerg','oncologist','prescript','m.d.','test','pass','prognosi','migran','seizur','c.t.',
#        'scan','imag','heartbeat','diagnosi','biopsi','remov','carcinoma','big c', 'live', 
#        'pass', 'radiat', 'kidney', 'chemo'],#1 Cancer
#       ['earth','sun','sky','star','tree','flower','land','sea','ocean','lake','river','rain','storm','thunder',
#        'lighten','snow','sunset','sunris','outsid','leaf','trek','wilder','anim','stream','rock','bolt','flood',
#        'weather','water','hurrican','morn','night','world','life','environ','landscap','view','cosmo','countri','forest',
#        'macrocosm','outdoor','sceneri','seascap','electr',
#        'set','univers','natur', 'cano', 'alaska', 'hike', 'disast', 'float',  
#        'forest', 'fire', 'soil', 'canyon', 'butterfli', 'camp', 
#        'gear', 'flower', 'mountain', 'hike', 'paddl', 'sunris', 'boat', 'backcountri'],#2 Nature
#       ['reflect','care','self car','health','consider','conscienti','regard','thought','thougt','well',
#        'well-b','feel','mental health','introspect','meditat','center','concentr','breath','relax','sit','focu',
#        'yoga','bodi','hardship','redifin','experi','sign','know','heal',
#        'posit','mind','deep','bodi','sens','tension','paus','notic','heartbeat',
#        'heart','check','self-esteem','confid','alert','care',
#        'concern','direct','forethougt',
#        'head','heed','interest','pain','regard',
#       'sens','breath','bodi','notic','posit','self-encourag','feel','reset','whole','chang',
#        'found','float','need','connect','let go','welcom','inspir','connect','creat','feast',
#        'feel','coma']]#3 Mindfullness

In [573]:
####actual guided topic model function and list of vocabulary from the transcripts###

from sklearn.feature_extraction.text import CountVectorizer
from lda import guidedlda as guidedlda
import numpy as np
model = guidedlda.GuidedLDA(n_topics=4,n_iter=1000,random_state=2022,refresh=10,alpha=0.01,eta=0.01) #alpha=0.01,eta=0.01

vectorizer = CountVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(text['processed'])

stemmed_missing =[]#list of words not found in the vocabulary
vocab = vectorizer.get_feature_names()
procab = vectorizer.get_feature_names()
word2id = dict((v,idx) for idx,v in enumerate(vocab))
seed_topics = {}

###connects words from the seed words list to the words in the vocabulary under one numeric I.D ###

for t_id, st in enumerate(seed_words):
    for word in st:
        try:
            seed_topics[word2id[word]] = t_id
        except:
            stemmed_missing.append(word)#adds words not found in vocab into stemmed_missing list
            print(word," skipped")

model.fit(X.toarray(),seed_topics=seedlist,seed_confidence=0.8) #set seed confidence to 0.8
topic_word = model.topic_word_
n_top_words = 20
vocab = tuple(vocab)

for i, topic_dist in enumerate(topic_word): #Print out results
    print('\n')
    print('Topic:',i)
    words_probability = np.array(-topic_dist)
    for index in range(n_top_words):
        print(round(abs(np.sort(words_probability))[:(n_top_words)][index],4),'*',
              np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1][index],sep='',end='  ') 

INFO:lda:n_documents: 30
INFO:lda:vocab_size: 7337
INFO:lda:n_words: 15336
INFO:lda:n_topics: 4
INFO:lda:n_iter: 1000


church  skipped
convict  skipped
optim  skipped
ideaolog  skipped
mass  skipped
synagogu  skipped
loyal  skipped
truth  skipped
assent  skipped
assur  skipped
credenc  skipped
credul  skipped
depend  skipped
fealti  skipped
relianc  skipped
sureti  skipped
troth  skipped
radiolog  skipped
benign  skipped
invas  skipped
malign  skipped
metastasi  skipped
melanoma  skipped
carcigen  skipped
precancer  skipped
survivor  skipped
diseas  skipped
ill  skipped
lump  skipped
cell  skipped
m.d.  skipped
migran  skipped
c.t.  skipped
carcinoma  skipped
big c  skipped
sky  skipped
sea  skipped
lake  skipped
lighten  skipped
snow  skipped
sunset  skipped
leaf  skipped
trek  skipped
wilder  skipped
stream  skipped
bolt  skipped
environ  skipped
landscap  skipped
view  skipped
cosmo  skipped
macrocosm  skipped
sceneri  skipped
seascap  skipped
univers  skipped
self car  skipped
consider  skipped
conscienti  skipped
regard  skipped
thougt  skipped
well-b  skipped
mental health  skipped
introspect  sk

/Users/lubasisikwibele/opt/anaconda3/lib/python3.9/site-packages/lda/guidedlda.py:303: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if w not in seed_topics:
/Users/lubasisikwibele/opt/anaconda3/lib/python3.9/site-packages/lda/guidedlda.py:318: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if w in seed_topics:
INFO:lda:<0> log likelihood: -176119
INFO:lda:<10> log likelihood: -156472
INFO:lda:<20> log likelihood: -155179
INFO:lda:<30> log likelihood: -154603
INFO:lda:<40> log likelihood: -154110
INFO:lda:<50> log likelihood: -153412
INFO:lda:<60> log likelihood: -152696
INFO:lda:<70> log likelihood: -152296
INFO:lda:<80> log likelihood: -151947
INFO:lda:<90> log likelihood: -151535
INFO:lda:<100> log likelihood: -151298
INFO:lda:<110> log likelihood: -151187
INFO:lda:<120> log likelihood: -151087
INFO:lda:<130> log likelihoo



Topic: 0
0.0558*speaker  0.0107*go  0.0096*know  0.0087*time  0.0059*fuck  0.0054*day  0.0045*cancer  0.0045*thing  0.0039*old  0.0039*told  0.0034*make  0.0034*lindsay  0.0034*well  0.0031*guy  0.0031*turn  0.0031*okay  0.0031*year old  0.0031*fish  0.0031*need  0.0028*move  

Topic: 1
0.0245*lindsey  0.0151*right  0.0085*guid  0.008*go  0.008*dad  0.0069*good  0.0069*roy  0.0066*ryan  0.0061*give  0.0058*phil  0.0055*okay  0.0052*lindsey dad  0.005*never  0.005*john  0.005*love  0.0047*guy  0.0047*time  0.0047*heart  0.0047*let  0.0047*mirror  

Topic: 2
0.0257*like  0.0167*get  0.0164*one  0.016*yeah  0.0147*speaker  0.0125*realli  0.0108*got  0.0105*come  0.0105*right  0.0099*see  0.009*think  0.0086*would  0.0083*back  0.0074*year  0.0072*littl  0.0064*water  0.0057*start  0.0057*peopl  0.0057*want  0.0053*someth  

Topic: 3
0.0103*elizabeth  0.0093*know  0.0067*kid  0.0063*breath  0.0063*day  0.0057*away  0.0057*kind  0.0043*take  0.0043*wait  0.004*feel  0.004*want  0.0037*act

# Company-Topic Heatmap
Based on proportion of each topic for each company

In [574]:
###this section creates the columns "topic number" and "category" in the data frame and loads the neccessary information

doc_topic = model.transform(X)
topic_number1 = []#list of topic numbers for each transcript in order

###connects relevant topic number and category to respective transcript name###

for i in range(len(doc_topic)):
    topic_number1.append(doc_topic[i].argmax())
text['topic number'] = [i for i in topic_number1]
text['category'] = [get_key(i) for i in text['topic number']]
text

,raw transcripts,transcript name,topic number,category,processed
0,Bow Knot - BRICK 2.mp4\n\nLindsey: [00:00:03] ...,Bow Knot - BRICK 2.mp4,1,Cancer,bow knot brick lindsey readi order custom gues...
1,On the rocks - Night rain - BRICK 0.mp4\n\nLar...,On the rocks - Night rain - BRICK 0.mp4,1,Cancer,rock night rain brick lar tnt guid hey guy goo...
2,9-26 Evening Hanging-out - BRICK.mp4\n\nSpeake...,9-26 Evening Hanging-out - BRICK.mp4,2,Nature,even hang brick speaker got bean pepper onion ...
3,Scene 1\n\n\n\nElizabeth: [00:00:12] I ended u...,Scene 1,3,Mindfullness,scene elizabeth end nine hour surgeri two mont...
4,Day before Wedding - BRICK 2.mp4\n\nLindsey: [...,Day before Wedding - BRICK 2.mp4,1,Cancer,day wed brick lindsey second ryan yeah lindsey...
5,ROER - BRICK 1.mp4\n\nSpeaker1: [00:00:03] I'l...,ROER - BRICK 1.mp4,2,Nature,roer brick speaker check radar go storm come a...
6,Home is a Place for Second Chances- BRICK.mp4\...,Home is a Place for Second Chances- BRICK.mp4,0,Faith,home place second chanc brick speaker big one ...
7,Wedding Day - BRICK 2.mp4\n\nSpeaker1: [00:00:...,Wedding Day - BRICK 2.mp4,0,Faith,wed day brick speaker ye sun shine bright good...
8,MD Anderson Day BRICK 3.2.mp4\n\nSpeaker1: [00...,MD Anderson Day BRICK 3.2.mp4,3,Mindfullness,anderson day brick speaker hear thunder awesom...
9,Scene 6\n\n\n\nMaynard: [00:26:28] I noticed w...,Scene 6,2,Nature,scene maynard notic water reflect light onto s...


In [559]:
###groups results by 'transcript name' and  'category'###
results = text[['transcript name', 'category']]

In [542]:
results

,transcript name,category
0,Bow Knot - BRICK 2.mp4,Cancer
1,On the rocks - Night rain - BRICK 0.mp4,Cancer
2,9-26 Evening Hanging-out - BRICK.mp4,Faith
3,Scene 1,Faith
4,Day before Wedding - BRICK 2.mp4,Nature
5,ROER - BRICK 1.mp4,Mindfullness
6,Home is a Place for Second Chances- BRICK.mp4,Nature
7,Wedding Day - BRICK 2.mp4,Nature
8,MD Anderson Day BRICK 3.2.mp4,Faith
9,Scene 6,Cancer


In [560]:
#####################################################################
#this section allows you to perform three main goals: 
#1) retrive transcripts based on genre 
#2) retrive transcripts based on transcript
#3) retrieve table results

In [562]:
###this function returns the transcripts for the input category (case sensitive!)###

def get_genre_scripts():
    category = input()
    return(text['transcript name'][text['category'] == str(category)].values)

In [544]:
get_genre_scripts()

 Nature


array(['9-26 Evening Hanging-out - BRICK.mp4', 'ROER - BRICK 1.mp4',
       'Scene 6', 'Campfire BRICK.mp4', 'Tattoo Tour - BRICK.mp4'],
      dtype=object)

In [563]:
###this function returns the category for the input transcript (case sensitive!)###

def get_script_genre():
    print('enter transcript name exactly as saved in file')
    transcriptinput = input()
    return(text['category'][text['transcript name'] == str(transcriptinput)].values)

In [546]:
get_script_genre()


enter transcript name exactly as saved in file


 Scene 12


array(['Cancer'], dtype=object)

In [566]:
# get_script_genre('transcript 5')

In [564]:
###this function prompts you to search for results based on transcript or category###

# def search():
#     print('enter transcripts or category to pick your search method')
#     method = input()
#     if method in 'transcripts':
#         get_script_genre()
#     else:
#         get_genre_scripts()

In [525]:
# search()

In [567]:
###this function returns based on choice the full results or search for a specific result in the data frame###

# def decision(choice):
#     if choice in 'table':
#         return results
#     else:
#         search()

In [568]:
### this function propmtis you to choose the full results or search for a specific result in the data frame###

# def start():
#     print('this is the transcript analyzer...', 'press enter to continue')
#     input()
#     print('enter table to get the results or enter search to find specific results')
#     choices = input()
#     decision(choices)

In [524]:
# start()